In [4]:
from Vissim_env_class import env

%matplotlib inline

In [5]:
model_name  = 'Balance'
vissim_working_directory =  'C:\\Users\\Rzhang\\Desktop\\MLforFlowOptimisationOrigine\\Vissim\\'
sim_length = 3601

# all controller actions
Controllers_Actions =\
{\
    # Controller Number 2 
    0 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 3
    1 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 4
    2 : {
            0 : [1,0,0,0,1,0,0,0],
            1 : [0,1,0,0,0,1,0,0],
            2 : [0,0,1,0,0,0,1,0],
            3 : [0,0,0,1,0,0,0,1]
        },
    # Controller Number 5
    3 : {
            0 : [1,0,0,1,0,0],
            1 : [0,1,0,0,1,0],
            2 : [0,0,1,0,0,1],
        },
    # Controller Number 6 
    4 : {
            0 : [1,1,0,0],
            1 : [0,1,1,0],
            2 : [1,0,0,1]
        },
    # Controller Number 8
    5 : {
            0 : [1,0,1],
            1 : [0,1,0]
        },
    # Controller Number 9
    6 : {
            0 : [1,0,1,0],
            1 : [0,1,0,1]
        },
    # Contoller Number 10
    7 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,1,0],
            2 : [0,0,1,0,1]
        },
    # Controller Number 12
    8 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller Number 13
    9 : {
            0 : [1,0,0,0],
            1 : [0,1,0,1],
            1 : [0,0,1,0],
        },
    # Controller 15
     10 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 16
    11 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 17
    12 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,0,1],
            2 : [0,0,1,0,0],
            3 : [0,0,0,1,1],
            4 : [0,1,0,0,1]
        },
    # Controller 33 
    13 : {
            0 : [1,0,0,1],
            1 : [1,0,1,0],
            2 : [0,1,0,0]
        }
}

In [6]:
env = env(model_name, vissim_working_directory, sim_length, Controllers_Actions,\
            timesteps_per_second = 1, mode = 'training', delete_results = True, verbose = True)

Working Directory set to: C:\Users\Rzhang\Desktop\MLforFlowOptimisationOrigine\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Balance.inpx ...
Load process successful
Simulation length set to 3601 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************

0.30044126510620117
0.2999255657196045
0.1878526210784912
0.2214345932006836
0.37449002265930176
0.39487743377685547
0.28858137130737305
0.30349206924438477
0.1109013557434082
0.12419819831848145
0.12379813194274902
0.14329814910888672
0.155

In [42]:
env.SCUs

In [43]:
env.SCUs[0].state

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [44]:
env.reset()